<h2> Remove stopwords </h2>  


In [1]:
import numpy as np
import pandas as pd
import os
import sys
import spacy


from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split



In [2]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [4]:
data_df = pd.read_csv('./data/rspct.tsv', sep='\t')

In [14]:
pre_processed = set() # Removed stopwords and alpha chars. Save in --> nostopwords_df
pos_processed = set() # PoS tag + remove stopwords
 
def pos_preprocess(text):
    # Returns PoS-tag for text
    doc = nlp(text)
    return [token.pos_ for token in doc]

def text_preprocess(text):
    # Returns PoS-tag for text
    doc = nlp(text)
    txt = [token.text for token in doc if (not token.is_stop or not token.is_alpha)]
    return " ".join(txt)

def preprocess(data_df):
    count = 0
    threshold = 10000
    print("Starting - Log every 10k parsed")

    for row in data_df.itertuples():
        title_processed = text_preprocess(row.title)
        selftext_processed = text_preprocess(row.selftext)
        feature_title = list(map(lambda x: "title_" + x, title_processed.split()))
        feature_text = list(map(lambda x: "body_" + x, selftext_processed.split()))
        #duplicates = [word for word in feature_title if "body_" + word.split("_")[0] in feature_text]
        
        #new_col = feature_title + feature_text + duplicates
        new_col = feature_title + feature_text

        new_col_stringify = " ".join(new_col)
        pre_processed.add((row.id, row.subreddit, new_col_stringify))
        
        if count >= threshold:
            print(count)
            threshold += 10000
        count+=1
def pos_parse(data_df):
    # Takes to long...
    count = 0
    threshold = 10000
    print("Starting - Log every 10k parsed")
    for row in data_df.itertuples():
        title_processed = text_preprocess(row.title)
        selftext_processed = text_preprocess(row.selftext)
        pos_title = pos_preprocess(title_processed)
        pos_selftext = pos_preprocess(selftext_processed)        
        #new_col = feature_title + feature_text + duplicates
        #new_col_stringify = " ".join(new_col)
        if count >= threshold:
            print(count)
            threshold += 10000
        pos_processed.add((row.id, row.subreddit, title_processed, " ".join(pos_title), selftext_processed, " ".join(pos_selftext)))
        count+=1

<h3> Create DF with removed stopwords and alpha and save to csv.</h3>

In [ ]:
preprocess(data_df)
processed_stopwords_df = pd.DataFrame(pre_processed, columns=("id", "subreddit", "body"))


In [ ]:
compression_opts = dict(method='zip',
                        archive_name='preprocess_stopwords.csv')  
processed_stopwords_df.to_csv('preprocess_stopwords.zip', index=False,
          compression=compression_opts)  

<h3> Extension on previous - Additionally add PoS-tag</h3>
Takes hours to run...

In [15]:
pos_parse(data_df)
processed_pos_df = pd.DataFrame(pos_processed, columns=("id", "subreddit", "title", "title_pos", "selftext", "selftext_pos"))


Starting - Log every 10k parsed
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000


In [16]:
compression_opts = dict(method='zip',
                        archive_name='preprocessed_stop_pos.csv')  
processed_pos_df.to_csv('preprocessed_stop_pos.zip', index=False,
          compression=compression_opts)  